# Linear Optimization with CVXPY

In [1]:
import numpy as np
import cvxpy as cp
print(cp.__version__) # version >= 1.5 is preferred

1.7.5


## Helper functions
We provide utilities to:
- store constraint information, 
- print solver results,
- print the left hand side and right hand side values of the constraints at a solution.

In [2]:
# Utility to create *named* constraints for later reporting.
# Each constraint is represented as (name, lhs_expr, sense, rhs_value, constraint_obj).
# sense ∈ {'<=', '>=', '=='}
def make_constraint(name, lhs, sense, rhs):
    if sense == '<=':
        constr = lhs <= rhs
    elif sense == '>=':
        constr = lhs >= rhs
    elif sense == '==':
        constr = lhs == rhs
    else:
        raise ValueError("Unknown sense")
    return (name, lhs, sense, rhs, constr)

# Print status, objective value and variables.
def print_results(problem, variables_dict=None, float_fmt="{:.6g}"):
    print(f"Status: {problem.status}")
    print(f"Objective value at optimum: {float_fmt.format(problem.value)}")
    if variables_dict:
        print("\nOptimal values:")
        for k, v in variables_dict.items():
            val = v.value
            if isinstance(val, np.ndarray):
                print(f"{k}: {val}")
            else:
                print(f"{k}: {float_fmt.format(val)}")

# Print the left-hand side value of constraints at the optimal solution.
def print_constraints(named_constraints, float_fmt="{:.6g}"):
    for (name, lhs, sense, rhs, constr) in named_constraints:
        lhs_val = lhs.value
        if isinstance(lhs_val, np.ndarray):
            lhs_val_str = str(lhs_val)
        else:
            lhs_val_str = float_fmt.format(lhs_val)
        print(f"{name}: LHS = {lhs_val_str}  {sense}  RHS = {rhs}")


## Example: A Linear Programming instance
Consider the following *linear maximization* problem:
\begin{aligned}
\text{maximize}\quad & x + 2y \\
\text{subject to}\quad & 2x + y \le 20, \\
& -4x + 5y \le 10, \\
& -x + 2y \ge -2, \\
& -x + 5y = 15, \\
& x \ge 0, \\
& y \ge 0.
\end{aligned}

In [ ]:
# Named variables and their domain
x = cp.Variable(nonneg=True, name='x') # x >= 0
y = cp.Variable(nonneg=True, name='y') # y >= 0

# Named constraints
named_constraints = []
named_constraints.append(make_constraint("constraint 1", 2*x + y, '<=', 20))
named_constraints.append(make_constraint("constraint 2", 4*x - 5*y, '>=', -10))
named_constraints.append(make_constraint("constraint 3", -x + 2*y, '>=', -2))
named_constraints.append(make_constraint("constraint 4", -x + 5*y, '==', 15))
constraints = [c[-1] for c in named_constraints]

# Objective
obj = cp.Maximize(x + 2*y)
prob = cp.Problem(obj, constraints)

# Run the solver and report results
prob.solve()
print_results(prob, variables_dict={"x": x, "y": y})
print_constraints(named_constraints)

## Example: A brewery problem


A small brewery produces ale and beer.

Monthly production limited by scarce resources: corn, hops, barley malt.

Recipes for ale and beer require different proportions of resources.


| Beverage      | Corn | Hops | Malt | Profit |
|:-------------:|:----:|:----:|:----:|:------:|
| Ale (barrel)  | 5    | 4    | 35   | 13     |
| Beer (barrel) | 15   | 4    | 20   | 23     |
| **Constraints** | 480  | 160  | 1190 | -      |

How many ale barrels and beer barrels should the brewery produce each month, if the goal is to maximize profit? 

In [ ]:
# Model
a = cp.Variable(nonneg=True, name='a')
b = cp.Variable(nonneg=True, name='b')

named_constraints = []
named_constraints.append(make_constraint("Corn constraint", 5*a + 15*b, '<=', 480))
named_constraints.append(make_constraint("Hops constraint", 4*a + 4*b, '<=', 160))
named_constraints.append(make_constraint("Malt constraint", 35*a + 20*b, '<=', 1190))
constraints = [c[-1] for c in named_constraints]

obj = cp.Maximize(13 * a + 23 * b)
prob = cp.Problem(obj, constraints)

prob.solve() 
print_results(prob, variables_dict={"Ale barrels": a, "Beer barrels": b})
print_constraints(named_constraints)